<a href="https://colab.research.google.com/github/j7tfj7f8k1f/DL-Fine-Tuning/blob/main/CLIP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 以下是普通的Fine Tuning CLIP模型的方式

## 前置作業HF_TOKEN = huggingface的金鑰





# 要達成文字圖片互相搜尋的clip模型需要一張圖搭配一組描述詞
# 比方睡衣吉伊卡哇.png + txt檔案內容為
# 裝飾 整體 是 穿著 粉紅 條紋 睡衣 和 頭戴 粉色 兔耳朵 頭飾 配有 棕色 小熊 身體配件 右手配件 是 粉色 毛線 球  表情 開心 毛 兩撇 嘴型 微笑
#它 是 睡衣吉伊卡哇

# 不一定要結巴 看任務需求

## <<步驟1>> 刪除文字檔跟資料夾不共存

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil

def check_and_delete_folder(folder_path):
    has_image = False
    has_text = False

    # 遍歷資料夾中的所有文件
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            # 檢查是否有圖片文件 (.jpg, .png)
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                has_image = True
            # 檢查是否有文字文件 (.txt)
            if file.lower().endswith('.txt'):
                has_text = True

    # 如果沒有圖片和文字檔共存，則刪除整個資料夾
    if not (has_image and has_text):
        shutil.rmtree(folder_path)
        print(f"已刪除資料夾: {folder_path}")
    else:
        print(f"保留資料夾: {folder_path}")

def process_folders(base_folder):
    # 遍歷所有子資料夾
    for root, dirs, files in os.walk(base_folder):
        for dir_name in dirs:
            folder_path = os.path.join(root, dir_name)
            check_and_delete_folder(folder_path)

# 指定要處理的資料夾路徑
base_folder = '/content/'
process_folders(base_folder)


保留資料夾: /content/3000/9371117346238611_ok
保留資料夾: /content/3000/1039097670999028_ok
保留資料夾: /content/3000/122112216074534901_ok
保留資料夾: /content/3000/2901772729996864_ok
保留資料夾: /content/3000/8738684762860230_ok
保留資料夾: /content/3000/27881267788127062_ok
保留資料夾: /content/3000/1238497030822243_ok
保留資料夾: /content/3000/577218687978885_ok
保留資料夾: /content/3000/122122546856400540_ok
保留資料夾: /content/3000/1190087632217619_ok
保留資料夾: /content/3000/513465254885967_ok
保留資料夾: /content/3000/2875900365897599_ok
保留資料夾: /content/3000/122120018798486044_ok
保留資料夾: /content/3000/27516479671276422_ok
保留資料夾: /content/3000/535562928881136_ok
保留資料夾: /content/3000/8176326059143662_ok
保留資料夾: /content/3000/9218936848133502_ok
保留資料夾: /content/3000/2432604416944075_ok
保留資料夾: /content/3000/8610303235695358_ok
保留資料夾: /content/3000/3817673035122884_ok
保留資料夾: /content/3000/3441120106189212_ok
保留資料夾: /content/3000/9459979670695327_ok
保留資料夾: /content/3000/3865975000305959_ok
保留資料夾: /content/3000/1075850813895120_ok_ok
保留資料夾: /

## <<步驟2>>合併資料夾內圖片與文字對應到csv方便訓練

In [ ]:
import os
import pandas as pd

# 資料夾路徑
base_path = "/content/"

# 儲存圖片與文字檔案的對應關係
image_text_pairs = []

# 遍歷所有資料夾
for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)

    if os.path.isdir(folder_path):
        # 找到所有的圖片與文字檔案
        images = [f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]
        text_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

        # 假設每個資料夾內的文字檔與所有圖片對應
        for img in images:
            for txt_file in text_files:
                with open(os.path.join(folder_path, txt_file), 'r', encoding='utf-8') as f:
                    text = f.read()
                image_text_pairs.append((os.path.join(folder_path, img), text))

# 將對應關係保存成CSV以備訓練
df = pd.DataFrame(image_text_pairs, columns=['image_path', 'text'])
df.to_csv('/content/image_text_pairs.csv', index=False)


## <<步驟3>>用CLIP processor先將圖片與文字轉為向量

In [ ]:
from transformers import CLIPProcessor, CLIPModel, ChineseCLIPProcessor, ChineseCLIPModel
from torch.utils.data import Dataset, DataLoader
import torch
from PIL import Image
import pandas as pd
import os

# 讀取預處理好的數據
df = pd.read_csv('/content/image_text_pairs.csv')

# 自定義 Dataset 類別
class CustomCLIPDataset(Dataset):
    def __init__(self, df, processor):
        self.df = df
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.df.iloc[idx]['image_path']
        text = self.df.iloc[idx]['text']

        # 讀取圖片
        try:
            image = Image.open(image_path).convert("RGB")  # 確保圖片為RGB格式
        except OSError:
            print(f"Skipping corrupted image: {image_path}")
            # Instead of returning None, return empty tensors with the correct shape
            # This ensures the batch has consistent data types
            input_ids = torch.zeros(128, dtype=torch.long)  # Replace 128 with your actual sequence length
            pixel_values = torch.zeros(3, 512, 512, dtype=torch.float32)  # Replace with your image dimensions
            attention_mask = torch.zeros(128, dtype=torch.long)  # Replace 128 with your actual sequence length
            return {"input_ids": input_ids, "attention_mask": attention_mask, "pixel_values": pixel_values}

        # 使用 CLIP processor 處理圖片和文字
        # Adding padding and truncation to ensure consistent input lengths
        inputs = self.processor(text=[text], images=image, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

        # 需要把 inputs 拆分成圖片和文字向量
        input_ids = inputs['input_ids'].squeeze(0)  # 文字向量
        pixel_values = inputs['pixel_values'].squeeze(0)  # 圖片向量
        attention_mask = inputs['attention_mask'].squeeze(0)  # 注意力掩碼

        return {"input_ids": input_ids, "attention_mask": attention_mask, "pixel_values": pixel_values}

# <<步驟4>>訓練CLIP模型

In [ ]:
# 載入繁體中文的 CLIP 模型和 processor
model = ChineseCLIPModel.from_pretrained("OFA-Sys/chinese-clip-vit-base-patch16")
processor = ChineseCLIPProcessor.from_pretrained("OFA-Sys/chinese-clip-vit-base-patch16")

# 構建數據集和數據加載器
dataset = CustomCLIPDataset(df, processor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 設定優化器和訓練參數
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 訓練參數
patience = 5  # 如果 5 個 epoch 沒有提升就提前停止
best_loss = float('inf')  # 記錄最好的損失值
counter = 0  # 沒有提升的 epoch 計數器
best_model_path = "best_clip_model.pth"  # 保存最佳模型的路徑

# 訓練過程
model.train()
for epoch in range(65):
    epoch_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()

        # 將數據移動到 GPU (如果可用)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)

        # 前向傳播
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)

        # 手動計算 contrastive loss (這裡假設為 text-image 對比損失)
        logits_per_image = outputs.logits_per_image  # 圖像與文本的 logits
        logits_per_text = outputs.logits_per_text  # 文本與圖像的 logits

        # 創建對比損失
        loss_fct = torch.nn.CrossEntropyLoss()
        batch_size = logits_per_image.shape[0]
        labels = torch.arange(batch_size, device=device)  # 自定義正確的標籤
        image_loss = loss_fct(logits_per_image, labels)
        text_loss = loss_fct(logits_per_text, labels)
        loss = (image_loss + text_loss) / 2

        # 反向傳播
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch + 1} completed. Average Loss: {avg_epoch_loss}")

    # 檢查是否為最佳損失
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        counter = 0  # 重置計數器
        torch.save(model.state_dict(), best_model_path)  # 保存最佳模型
        print(f"Best model saved at epoch {epoch + 1} with loss {best_loss}")
    else:
        counter += 1  # 沒有提升，增加計數

    # 如果連續 5 個 epoch 沒有提升，停止訓練
    if counter >= patience:
        print(f"Training stopped early at epoch {epoch + 1} due to no improvement for {patience} epochs.")
        break


config.json:   0%|          | 0.00/3.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/753M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ChineseCLIPImageProcessor.preprocess` and were ignored: 'padding', 'truncation', 'max_length'
  return self.preprocess(images, **kwargs)


Epoch 1 completed. Average Loss: 2.5098253802249304
Best model saved at epoch 1 with loss 2.5098253802249304
Epoch 2 completed. Average Loss: 0.7209921902731845
Best model saved at epoch 2 with loss 0.7209921902731845
Epoch 3 completed. Average Loss: 0.31366780282635437
Best model saved at epoch 3 with loss 0.31366780282635437
Epoch 4 completed. Average Loss: 0.14113252413900276
Best model saved at epoch 4 with loss 0.14113252413900276
Epoch 5 completed. Average Loss: 0.10388498331763242
Best model saved at epoch 5 with loss 0.10388498331763242
Epoch 6 completed. Average Loss: 0.07538444066028062
Best model saved at epoch 6 with loss 0.07538444066028062
Epoch 7 completed. Average Loss: 0.07518117867508217
Best model saved at epoch 7 with loss 0.07518117867508217
Epoch 8 completed. Average Loss: 0.09088836014760952
Epoch 9 completed. Average Loss: 0.05826550352044011
Best model saved at epoch 9 with loss 0.05826550352044011
Epoch 10 completed. Average Loss: 0.036786810936112156
Best mod

# 不滿意繼續訓練

In [ ]:
import time  # 引入 time 模組來計時
import torch

# 載入之前保存的最佳模型權重
best_model_path = "best_clip_model.pth"
model.load_state_dict(torch.load(best_model_path))

# 確保模型仍在 GPU 上
model.to(device)

# 設定優化器和學習率
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# 訓練參數
patience = 5  # 如果 5 個 epoch 沒有提升就提前停止
best_loss = float('inf')  # 如果有需要，載入之前最好的損失值
counter = 0  # 沒有提升的 epoch 計數器
start_epoch = 65  # 從之前的 epoch 繼續訓練

# 訓練過程
model.train()
for epoch in range(start_epoch, start_epoch + 65):  # 繼續從 65 開始
    epoch_loss = 0
    start_time = time.time()  # 記錄 epoch 開始的時間
    total_batches = len(dataloader)

    for batch_idx, batch in enumerate(dataloader):
        batch_start_time = time.time()  # 記錄 batch 開始的時間
        optimizer.zero_grad()

        # 將數據移動到 GPU (如果可用)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)

        # 前向傳播
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)

        # 手動計算 contrastive loss
        logits_per_image = outputs.logits_per_image  # 圖像與文本的 logits
        logits_per_text = outputs.logits_per_text  # 文本與圖像的 logits

        # 創建對比損失
        loss_fct = torch.nn.CrossEntropyLoss()
        batch_size = logits_per_image.shape[0]
        labels = torch.arange(batch_size, device=device)  # 自定義正確的標籤
        image_loss = loss_fct(logits_per_image, labels)
        text_loss = loss_fct(logits_per_text, labels)
        loss = (image_loss + text_loss) / 2

        # 反向傳播
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # 計算 batch 完成的時間
        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # 計算剩餘 batch 的時間
        remaining_batches = total_batches - (batch_idx + 1)
        remaining_time = remaining_batches * batch_time

        # 格式化剩餘時間
        remaining_mins, remaining_secs = divmod(remaining_time, 60)

        print(f"Batch {batch_idx + 1}/{total_batches} completed. Estimated remaining time for epoch: {int(remaining_mins)} minutes {int(remaining_secs)} seconds.")

    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch + 1} completed. Average Loss: {avg_epoch_loss}")

    # 檢查是否為最佳損失
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        counter = 0  # 重置計數器
        torch.save(model.state_dict(), best_model_path)  # 保存最佳模型
        print(f"Best model saved at epoch {epoch + 1} with loss {best_loss}")
    else:
        counter += 1  # 沒有提升，增加計數

    # 如果連續 5 個 epoch 沒有提升，停止訓練
    if counter >= patience:
        print(f"Training stopped early at epoch {epoch + 1} due to no improvement for {patience} epochs.")
        break


# 保存到雲端

In [ ]:
# 保存模型到指定路徑
model_save_path = "/content/drive/MyDrive/chiikawa_chinese_clip_model.pth"

# 保存模型權重
torch.save(model.state_dict(), model_save_path)

print(f"Model saved to {model_save_path}")


Model saved to /content/chinese_clip_model.pth


## 完成 灑花 🖖